In [1]:
.libPaths(c('/root/miniconda3/envs/metadoc/lib/R/library','/root/miniconda3/envs/Rdoc/lib/R/library'))

In [2]:
setwd('/import/hd/')

In [3]:
library(SPOTlight)
library(Seurat)
library(ggplot2)
library(SingleCellExperiment)
library(scater)
library(scran)

Warning message:
"package 'Seurat' was built under R version 4.2.3"
Attaching SeuratObject

Loading required package: SummarizedExperiment

Warning message:
"package 'SummarizedExperiment' was built under R version 4.2.2"
Loading required package: MatrixGenerics

Loading required package: matrixStats

Warning message:
"package 'matrixStats' was built under R version 4.2.3"

Attaching package: 'matrixStats'


The following objects are masked from 'package:Biobase':

    anyMissing, rowMedians



Attaching package: 'MatrixGenerics'


The following objects are masked from 'package:matrixStats':

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
   

In [4]:
sc = readRDS('./sc/scrna.rds')

In [5]:
ST = readRDS('./N2.rds')

In [13]:
sce <- as.SingleCellExperiment(sc)
### Feature selection
sce <- logNormCounts(sce)
### Variance modelling
# 去掉核糖体和线粒体基因
genes <- !grepl(pattern = "^RP[L|S]|MT", x = rownames(sce))
dec <- modelGeneVar(sce , subset.row = genes)

In [14]:
dec

DataFrame with 36221 rows and 6 columns
                   mean       total        tech         bio   p.value       FDR
              <numeric>   <numeric>   <numeric>   <numeric> <numeric> <numeric>
MIR1302-2HG  0.00000000  0.00000000  0.00000000  0.0000e+00       NaN       NaN
FAM138A      0.00000000  0.00000000  0.00000000  0.0000e+00       NaN       NaN
OR4F5        0.00000000  0.00000000  0.00000000  0.0000e+00       NaN       NaN
AL627309.1   0.00124591  0.00143283  0.00137843  5.4404e-05  0.464848  0.675528
AL627309.3   0.00000000  0.00000000  0.00000000  0.0000e+00       NaN       NaN
...                 ...         ...         ...         ...       ...       ...
AC141272.1  0.000000000 0.000000000 0.000000000 0.00000e+00       NaN       NaN
AC023491.2  0.000000000 0.000000000 0.000000000 0.00000e+00       NaN       NaN
AC007325.1  0.000000000 0.000000000 0.000000000 0.00000e+00       NaN       NaN
AC007325.4  0.030567394 0.034940993 0.033818709 1.12228e-03  0.470432  0.675528


In [7]:
hvg <- getTopHVGs(dec, n = 3000)

# 加上细胞注释信息
colLabels(sce) <- colData(sce)$celltype3

# Compute marker genes
mgs <- scoreMarkers(sce, subset.row = genes)

# 保留最相关的marker基因
mgs_fil <- lapply(names(mgs), function(i) {
  x <- mgs[[i]]
  # Filter and keep relevant marker genes, those with AUC > 0.8
  x <- x[x$mean.AUC > 0.8, ]
  # Sort the genes from highest to lowest weight
  x <- x[order(x$mean.AUC, decreasing = TRUE), ]
  # Add gene and cluster id to the dataframe
  x$gene <- rownames(x)
  x$cluster <- i
  data.frame(x)
})
mgs_df <- do.call(rbind, mgs_fil)

Warning message:
"useNames = NA is deprecated. Instead, specify either useNames = TRUE or useNames = TRUE."
Warning message:
"useNames = NA is deprecated. Instead, specify either useNames = TRUE or useNames = TRUE."
Warning message:
"useNames = NA is deprecated. Instead, specify either useNames = TRUE or useNames = TRUE."
Warning message:
"useNames = NA is deprecated. Instead, specify either useNames = TRUE or useNames = TRUE."
Warning message:
"useNames = NA is deprecated. Instead, specify either useNames = TRUE or useNames = TRUE."
Warning message:
"useNames = NA is deprecated. Instead, specify either useNames = TRUE or useNames = TRUE."
Warning message:
"useNames = NA is deprecated. Instead, specify either useNames = TRUE or useNames = TRUE."
Warning message:
"useNames = NA is deprecated. Instead, specify either useNames = TRUE or useNames = TRUE."
Warning message:
"useNames = NA is deprecated. Instead, specify either useNames = TRUE or useNames = TRUE."
Warning message:
"useNames =

In [11]:
packageVersion('Seurat')

[1] '4.3.0.1'

In [9]:
Seurat::Assays(ST)

[1] "Spatial" "SCT"

In [10]:
res <- SPOTlight(
  x = sce, 
  y = ST,
  groups = as.character(sc$celltype3), # 也可以是cluster，
  mgs = mgs_df,
  hvg = hvg,assay_sp = "Spatial",
  weight_id = "mean.AUC",
  group_id = "cluster",
  gene_id = "gene")


Warning message in asMethod(object):
"sparse->dense coercion: allocating vector of size 10.3 GiB"


ERROR: Error in .extract_counts(y, assay, slot): assay %in% SeuratObject::Assays(x) is not TRUE


In [ ]:

  
#Extract data from `SPOTlight`:
str(res) #查看结果类型 
decon_mtrx <- res$mat
#这里重命名，非必要
colnames(decon_mtrx) <- paste(colnames(decon_mtrx),"Spotlight",sep = "_")